In [16]:
#Automatic film scanning for Film Carriers
#Using Arduino Nano
#MIT License
#Copyright (c) 2020 Seckin Sinan Isik

import cv2, time
import numpy as np
import pyfirmata
from scipy.stats import rankdata

In [17]:
board = pyfirmata.Arduino('/dev/cu.usbserial-14130')
it = pyfirmata.util.Iterator(board)
it.start()
Shutter    = board.get_pin('d:12:o')
Shutter.write(1) #starts with a 0 (False) on Arduino Nano
motMS1Pin  = board.get_pin('d:11:o')
motMS2Pin  = board.get_pin('d:10:o')
motMS3Pin  = board.get_pin('d:9:o')
motStepPin = board.get_pin('d:8:o')
motDirPin  = board.get_pin('d:7:o')
buttonRev     = board.get_pin('d:4:i')
buttonFrw     = board.get_pin('d:5:i')
buttonRun     = board.get_pin('d:6:i')
buttonPicture = board.get_pin('d:3:i')
laserInt      = board.get_pin('a:5:i')

In [18]:
MaxPicutureNumber = 40
idealBorderSize   = 7  # usually is around 7 pixels when image is shrunk to 150:100 pixels
BorderExpectation = 10 # initial move is (x-1)/x th of the frame, 10 is ideal
BackupPictureTake = 1  # 1 for yes, 0 for no  
waitingTime       = 1  # 1 seconds after shuter is relesed, If blury pictures increase

In [19]:
CorrectionSpeed = 1776 # resolution of a 1/16 microstepping  per frame
FrameLength     = 111  # resolution of a 1/1 normal stepping per frame

In [20]:
print("lets go")
latch           = 0    # used for second border searching
start           = 0 
counter         = 1    # picutre counter
autoAuto        = 0
BadFrames       = 0
while True:
    ###########################################
    ################ Functions ################
    def my_label(res):
        label=np.zeros(len(res))
        label[0]=1
        for i in range(len(res)-1):            
            if res[i]==res[i+1]-1:
                label[i+1]=label[i]
            else:
                label[i+1]=label[i]+1
        return label
    def my_takeApicuture(counter):
        print("picture number:", counter)
        counter=counter+1
        Shutter.write(0)
        time.sleep(0.3) #0.045 is able to trigger the shutter on a xt2 
        Shutter.write(1)
        time.sleep(waitingTime)
        return counter
        
    def my_functionFAST(xx,yy,a,b,c,wait):
        motDirPin.write(xx)
        motMS1Pin.write(a)
        motMS2Pin.write(b)
        motMS3Pin.write(c)            
        for x in range(yy):
            motStepPin.write(1)
            time.sleep(wait)
            motStepPin.write(0)
            time.sleep(wait)

    def my_maxEdge(xFrameProjection,position): 
        #set_trace()
        if min(position)-2 >= 0:
            a=float(xFrameProjection[min(position)])
            b=float(xFrameProjection[min(position)-2] )
            left=abs(a-b)
            #print("left is:",left)
        if max(position)+2 <= 148:
            a=float(xFrameProjection[max(position)])
            b=float(xFrameProjection[max(position)+2])
            right=abs(a-b)
            #print("right is:",right)
        if min(position) <= 2 or max(position) >= 146:
            return -1,-1
        else:
            return left,right
            
    def my_captureData():
        video=cv2.VideoCapture(1) #this could be 0 or 2 depending on your available inputs, such as webcam
        time.sleep(0.5)
        check, frame=video.read() #capturing screen 
        frame= cv2.resize(frame, (178, 100), interpolation=cv2.INTER_LINEAR) #downsizing 
        height=len(frame[:,0])
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        lv= frame[0,:]>0
        
        #getting rid of the extra sides from the screen capture
        CropVertical=round(height*0.2) #cut out the effect of sprocket holes
        frame=frame[0+CropVertical:height-CropVertical, min([i for i, x in enumerate(lv) if x]):max([i for i, x in enumerate(lv) if x])]
        #creating candidate borders
        maskHor = (np.sum(frame,axis=0)/max(np.sum(frame,axis=0))>0.95)*1
        maskVer = (np.sum(frame,axis=1)/max(np.sum(frame,axis=1))>0.95)*1 #not usuing, may delete   
        #labeling candidate borders
        position= np.where(maskHor==1)[0]    
        items=my_label(position)
        xFrameProjection=np.sum(frame,axis=0)
        #bringing data together
        a=np.array([( x, 
                     len(position[items==x]),
                     round(np.median(position[items==x])),
                     max(np.sum(frame,axis=0)[position[items==x]])            
                    ) 
            for i, x in enumerate(np.unique(items))]) 
        # my_maxEdge(xFrameProjection,position[items==x])
        return a,xFrameProjection,position,items,maskHor,frame
    
    def my_CheckingEdgeDeciding(aa,xFrameProjection,position,items,latch,BackupPictureTake):
        aa[:,1]=abs(aa[:,1]-idealBorderSize)
        Positionselected=0
        weights=[[1],[4],[1]]
        if   latch ==  0:  #FORWARD expected border
            aa[:,2]=abs(aa[:,2]-round(len(frame[0,:])/BorderExpectation))
        elif latch == 2:  #BACKWARD expected border
            aa[:,2]=abs(aa[:,2]-(BorderExpectation-1.5)/BorderExpectation*len(frame[0,:]))
            
        decide= sum ( [rankdata(-aa[:,1]),rankdata(-aa[:,2]),rankdata(aa[:,3])]*np.array(weights) ) #applying decision weights 
        Itemselected=aa[decide==max(decide),0]                                #selected pulse as border
        # if the Selected border has a width is >1, conatains informationm and there is one selected border
        if len(Itemselected)==1 and len(position[items==Itemselected])!=1:
            left,right=my_maxEdge(xFrameProjection,position[items==Itemselected])
            if left>5*right and len(Itemselected)==1 and ( right>300 or left>300 ):     #prominant edge on the left
                #print("Prominent-L")
                Positionselected=min(position[items==Itemselected])+2
                if latch==0:
                    latch=0
                elif latch==2:
                    latch=1
            elif right>5*left and len(Itemselected)==1 and ( right>300 or left>300 ):  #prominant edge on the right
                #print("Prominent-R")
                Positionselected=max(position[items==Itemselected])-2
                if latch==0:
                    latch=0
                elif latch==2:
                    latch=1
            elif right>500 and left>500:                                         #there is some edge on both side
                #print("okEdge")
                Positionselected=round(np.median(position[items==Itemselected]))   
                if latch==0:
                    latch=0
                elif latch==2:
                    latch=1
            elif right==-1 and left==-1:
                if latch==0:
                    print("FrameEdge not clear")
                    latch=2          #repeat the postion finding looop
                elif BackupPictureTake==1 and latch==2:
                    latch=3
                elif BackupPictureTake==0 and latch==2:
                    latch=4  
            else:
                #print("Poor PulseEdge")
                #set_trace()
                if latch==0:
                    latch=2          #repeat the postion finding looop
                elif BackupPictureTake==1 and latch==2:
                    latch=3      
                elif BackupPictureTake==0 and latch==2:
                    latch=4  
        else:
            print("Poor PulseEdge")
            if latch==0:
                latch=2          #repeat the postion finding looop
            elif BackupPictureTake==1 and latch==2:
                latch=3
            elif BackupPictureTake==0 and latch==2:
                latch=4  
    
        return latch,Positionselected        
                      
    ###########################################
    ################## Main LOOP ############### 
    FullAuto                     = board.digital[6].read()
    FrameByFrameWithPicture      = board.digital[5].read()
    FrameByFrame                 = board.digital[4].read()
    ManualshutterButton          = board.digital[3].read()
        
    if start==1 or autoAuto==3 or latch>=1:
        skip=0 
        # 1-Start with taking a picture
        if counter==1 and start>=2:
            counter=my_takeApicuture(counter)
            time.sleep(1)
        # 2-Initial film advancement
        if latch==0: 
            my_functionFAST(1,round((BorderExpectation-1)/BorderExpectation*FrameLength),0,0,0,0.0008)
        elif latch==2:
            my_functionFAST(1,round((1+1.5)/BorderExpectation*FrameLength),0,0,0,0.0008)

        # 3-Capture FRAME to evaluate     
        a,xFrameProjection,position,items,maskHor,frame = my_captureData() 
        # 4-Evaluating preliminary data and checking the features of the selected data
        latch,Positionselected=my_CheckingEdgeDeciding(a,xFrameProjection,position,items,latch,BackupPictureTake)
        # 5-Borders are not found twice, but the user wants to scan the entire film strip
        #set_trace()
        if latch >=3:   
            if latch==3:
                my_functionFAST(0,round((2.5-1)/BorderExpectation*FrameLength),0,0,0,0.0008)
                counter=my_takeApicuture(counter)
                BadFrames=BadFrames+1
            elif latch==4:
                my_functionFAST(0,round((2.5-1)/BorderExpectation*FrameLength),0,0,0,0.0008)
            latch=0
            skip=1
        # 6-STOP if Picture Number>x and there is no information
        if MaxPicutureNumber<counter: #do not excede maximum picture numper
            if np.std(xFrameProjection)<500:
                set_trace()
                print("scan complete")
                print("number of BAD scans:", BadFrames)
                print("number of scans:", counter)
                latch=0
             
        # 7-Correction PART and taking picture
        if skip==0:
            if latch==0:
                my_functionFAST(1,round(Positionselected/len(maskHor)*CorrectionSpeed),1,1,1,0.00008)
                if autoAuto>=2:
                    counter=my_takeApicuture(counter)
            elif latch==1:
                my_functionFAST(0,round( (len(maskHor)-Positionselected+2) /len(maskHor)*CorrectionSpeed),1,1,1,0.00008)
                latch=0
                if autoAuto>=2:
                    counter=my_takeApicuture(counter)
        else:
            skip=0
        # 8-Stop FrameByFrameWithPicture and FrameByFrame
        start=0   
            
    if  FullAuto     is False:
        autoAuto=3
        time.sleep(0.2)
    if FrameByFrameWithPicture is False:
        autoAuto=2
        start=1
        time.sleep(0.2)
    if FrameByFrame is False:
        autoAuto=1
        start=1
        time.sleep(0.2)
    if ManualshutterButton is False:  
        counter=my_takeApicuture(counter)


lets go


KeyboardInterrupt: 